In [6]:
# Carregando pacotes
import pandas as pd
import numpy as np

from sklearn import preprocessing

from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import confusion_matrix, roc_curve, accuracy_score, balanced_accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.inspection import DecisionBoundaryDisplay

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from IPython.display import HTML

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV,HalvingGridSearchCV

In [2]:
# Carregando dados
original = pd.read_csv("dados.csv")
le = preprocessing.LabelEncoder()

modificado = pd.DataFrame()
for x in original:
    if original[x].dtypes==object:
        modificado[x] = le.fit_transform(np.array(original[x]))
    else:
        modificado[x] = np.array(original[x])


y = modificado.COMPARECEU_AGENDAMENTO_EXAME
X = modificado.drop('COMPARECEU_AGENDAMENTO_EXAME', axis=1)

Xd,y = np.array(X), 1-np.array(y)
X = (Xd-Xd.min(axis=0))/(Xd.max(axis=0)-Xd.min(axis=0))

In [5]:
classificadores=[("rede neural",MLPClassifier()),("VM não linear",SVC(gamma="auto",probability=True,class_weight='balanced'))]

In [7]:
stm = SVC(gamma="auto",probability=True,class_weight='balanced')
param_grid = {'C': [0.1, 0.5, 1,5,10,50,100]}
sh = HalvingGridSearchCV(stm, param_grid, cv=5).fit(X, y)

In [11]:
stm = SVC(gamma="auto",probability=True,class_weight='balanced',C=100)
X_train, X_test, y_train, y_test = train_test_split(Xd, y, test_size=0.25)
stm.fit(X_train,y_train)

SVC(C=100, class_weight='balanced', gamma='auto', probability=True)

In [36]:
y_tp = stm.predict_proba(X_test)
xxx = confusion_matrix(y_test,(y_tp[:,1]>0.06854).astype(bool),normalize='true')
xxx



array([[0.3876205, 0.6123795],
       [0.488    , 0.512    ]])

In [9]:
pd.DataFrame(sh.cv_results_)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0,2794,1.698517,0.068427,0.152577,0.008344,0.1,{'C': 0.1},0.629696,0.620751,...,0.601344,0.029826,10,0.558389,0.570917,0.575839,0.624161,0.633557,0.592573,0.030315
1,0,2794,1.558834,0.070274,0.147501,0.039805,0.5,{'C': 0.5},0.681574,0.624329,...,0.613152,0.044239,9,0.627740,0.590604,0.584787,0.618792,0.599105,0.604206,0.016460
2,0,2794,1.628885,0.152259,0.141201,0.024484,1,{'C': 1},0.690519,0.626118,...,0.617091,0.046978,8,0.647427,0.600000,0.595973,0.627740,0.604922,0.615213,0.019505
3,0,2794,1.427137,0.034082,0.121713,0.009403,5,{'C': 5},0.674419,0.688730,...,0.658293,0.024691,5,0.666667,0.693065,0.654586,0.679195,0.656823,0.670067,0.014414
4,0,2794,1.566893,0.087568,0.123666,0.004802,10,{'C': 10},0.677996,0.665474,...,0.662242,0.017226,4,0.672036,0.687248,0.694407,0.702013,0.666667,0.684474,0.013310
5,0,2794,1.491589,0.021671,0.109707,0.002386,50,{'C': 50},0.681574,0.633274,...,0.667271,0.019889,2,0.686353,0.703803,0.724832,0.709620,0.723937,0.709709,0.014223
6,0,2794,1.677936,0.130869,0.127712,0.022246,100,{'C': 100},0.704830,0.645796,...,0.680519,0.020828,1,0.708277,0.718121,0.731096,0.717673,0.754810,0.725996,0.016134
7,1,8382,16.085077,0.402522,1.125538,0.020604,10,{'C': 10},0.701849,0.666667,...,0.662725,0.025689,3,0.683967,0.686652,0.676260,0.682076,0.674023,0.680596,0.004738
8,1,8382,17.059134,0.381652,1.080373,0.021738,50,{'C': 50},0.668456,0.653548,...,0.650320,0.017323,6,0.666816,0.675764,0.677453,0.675067,0.674023,0.673825,0.003678
9,1,8382,18.360901,0.311221,1.066293,0.017046,100,{'C': 100},0.662493,0.647585,...,0.645428,0.020055,7,0.671588,0.683669,0.686698,0.678497,0.685506,0.681192,0.005560
